# Decorators

Decorators are **syntactic sugar** that lets you **wrap code** to a **function**.

![decorators](../images/decorators.png)

## Decorators in BioSTEAM
We can use decorators to create input **parameters** and output **indicators** to an uncertainty/sensitivity **model**. 

In [1]:
import biosteam as bst
from biorefineries import cellulosic
bst.nbtutorial()
br = cellulosic.Biorefinery()
br

Biorefinery(
    # dry at 20% moisture content
    feedstock='cornstover',
    # final product
    product='ethanol',
    # whether to model boiler/cooling tower blowdown to wastewater
    include_blowdown_recycle=False,
)


In [2]:
br.model

Model:
parameters: None
indicators: None


In [4]:
@br.model.indicator(units='USD/kg')
def MESP(): 
    return br.tea.solve_price(bst.F.ethanol)

MESP

<Indicator: MESP (USD/kg)>

In [5]:
MESP()

0.6923643244026552

In [6]:
br.model

Model:
parameters: None
indicators: MESP [USD/kg]


## Basics
Let's make a decorator that adds 1 to the result of a function.

In [1]:
# The function
def add_numbers(a, b):
    return a + b

add_numbers(1, 2)

3

In [4]:
# The decorator
def plus_one_decorator(f):
    def wrapped_f(a, b):
        return f(a, b) + 1
    return wrapped_f

# The decorated function
add_numbers = plus_one_decorator(add_numbers)
add_numbers(1, 2)

4

In [5]:
def plus_one_decorator(f):
    def wrapped_f(a, b):
        return f(a, b) + 1
    return wrapped_f

# Syntactic sugar
@plus_one_decorator
def add_numbers(a, b):
    return a + b

add_numbers(1, 2)

4

We can also use decorators to perform **supporting actions** without changing the function output. Let's register functions in a list.

In [7]:
parameters_list = []

def register(f):
    function_list.append(f)
    return f

@register
def function(x):
    return None

function_list

[<function __main__.function(x)>]

Now let's add **metadata** by adding **arguments** to the decorator.

In [13]:
function_dct = {}
def register(f=None, name=None):
    if f is None: return lambda f: register(f, name)
    function_dct[name] = f
    return f

@register(name='A')
def function_A(x):
    return None

@register(name='B')
def function_B(x):
    return None

function_dct

{'A': <function __main__.function_A(x)>,
 'B': <function __main__.function_B(x)>}

Let's contain the metadata in a **callable object** instead.

In [17]:
objs = []

class Indicator:
    def __init__(self, f, name):
        self.name = name
        self.f = f

    def __call__(self):
        return self.f()

    def __repr__(self):
        return f"Indicator({self.f.__name__}, {self.name!r})" 

def register(f=None, name=None):
    if f is None: return lambda f: register(f, name)
    objs.append(Indicator(f, name))
    return f

@register(name='A')
def function_A():
    return 1

@register(name='B')
def function_B():
    return 2

objs

[Indicator(function_A, 'A'), Indicator(function_B, 'B')]

In [18]:
[indicator() for indicator in objs]

[1, 2]